In [1]:
import tensorflow as tf
import numpy as np
import os
import time
import datetime
import data_helpers
from text_cnn import TextCNN
from tensorflow.contrib import learn

In [2]:
#Define Parameters
tf.flags.DEFINE_float("dev_sample_percentage", .05, "Percentage of the training data to use for validation")
tf.flags.DEFINE_integer("embedding_dim", 128, "Dimensionality of character embedding (default: 128)")
tf.flags.DEFINE_string("filter_sizes", "3,4,5", "Comma-separated filter sizes (default: '3,4,5')")
tf.flags.DEFINE_integer("num_filters", 128, "Number of filters per filter size (default: 128)")
tf.flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")
tf.flags.DEFINE_float("l2_reg_lambda", 0.0, "L2 regularization lambda (default: 0.0)")
# Training parameters
tf.flags.DEFINE_integer("batch_size", 32, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("num_epochs", 100, "Number of training epochs (default: 200)")
tf.flags.DEFINE_integer("evaluate_every", 100, "Evaluate model on dev set after this many steps (default: 100)")
tf.flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
tf.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")
# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")

FLAGS = tf.flags.FLAGS
FLAGS._parse_flags()
print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")

positive_data_file = ''
negative_data_file = ''



Parameters:
ALLOW_SOFT_PLACEMENT=True
BATCH_SIZE=32
CHECKPOINT_EVERY=100
DEV_SAMPLE_PERCENTAGE=0.05
DROPOUT_KEEP_PROB=0.5
EMBEDDING_DIM=128
EVALUATE_EVERY=100
FILTER_SIZES=3,4,5
L2_REG_LAMBDA=0.0
LOG_DEVICE_PLACEMENT=False
NUM_CHECKPOINTS=5
NUM_EPOCHS=100
NUM_FILTERS=128



In [3]:
def train(classId, positive_data_file, negative_data_file):
    # Data Preparation
    # ==================================================

    # Load data
    print("Loading data...")
    x_text, y = data_helpers.load_data_and_labels(positive_data_file, negative_data_file)

    # Build vocabulary
    max_document_length = max([len(x.split(" ")) for x in x_text])
    vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
    x = np.array(list(vocab_processor.fit_transform(x_text)))

    # Randomly shuffle data
    np.random.seed(10)
    shuffle_indices = np.random.permutation(np.arange(len(y)))
    x_shuffled = x[shuffle_indices]
    y_shuffled = y[shuffle_indices]

    # Split train/test set
    # TODO: This is very crude, should use cross-validation
    dev_sample_index = -1 * int(FLAGS.dev_sample_percentage * float(len(y)))
    x_train, x_dev = x_shuffled[:dev_sample_index], x_shuffled[dev_sample_index:]
    y_train, y_dev = y_shuffled[:dev_sample_index], y_shuffled[dev_sample_index:]
    print("Vocabulary Size: {:d}".format(len(vocab_processor.vocabulary_)))
    print( vocab_processor.vocabulary_)
    print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))
    
# Training
    # ==================================================

    with tf.Graph().as_default():
        session_conf = tf.ConfigProto(
          allow_soft_placement=FLAGS.allow_soft_placement,
          log_device_placement=FLAGS.log_device_placement)
        sess = tf.Session(config=session_conf)
        with sess.as_default():
            cnn = TextCNN(
                sequence_length=x_train.shape[1],
                num_classes=y_train.shape[1],
                vocab_size=len(vocab_processor.vocabulary_),
                embedding_size=FLAGS.embedding_dim,
                filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
                num_filters=FLAGS.num_filters,
                l2_reg_lambda=FLAGS.l2_reg_lambda)

            # Define Training procedure
            global_step = tf.Variable(0, name="global_step", trainable=False)
            optimizer = tf.train.AdamOptimizer(1e-3)
            grads_and_vars = optimizer.compute_gradients(cnn.loss)
            train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

            # Keep track of gradient values and sparsity (optional)
            grad_summaries = []
            for g, v in grads_and_vars:
                if g is not None:
                    grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                    sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                    grad_summaries.append(grad_hist_summary)
                    grad_summaries.append(sparsity_summary)
            grad_summaries_merged = tf.summary.merge(grad_summaries)

            # Output directory for models and summaries
            timestamp = str(int(time.time()))
            out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", str(classId), timestamp))
            print("Writing to {}\n".format(out_dir))

            # Summaries for loss and accuracy
            loss_summary = tf.summary.scalar("loss", cnn.loss)
            acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

            # Train Summaries
            train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
            train_summary_dir = os.path.join(out_dir, "summaries", "train")
            train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

            # Dev summaries
            dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
            dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
            dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

            # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
            checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
            checkpoint_prefix = os.path.join(checkpoint_dir, "model")
            if not os.path.exists(checkpoint_dir):
                os.makedirs(checkpoint_dir)
            saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

            # Write vocabulary
            vocab_processor.save(os.path.join(out_dir, "vocab"))

            # Initialize all variables
            sess.run(tf.global_variables_initializer())

            def train_step(x_batch, y_batch):
                """
                A single training step
                """
                feed_dict = {
                  cnn.input_x: x_batch,
                  cnn.input_y: y_batch,
                  cnn.dropout_keep_prob: FLAGS.dropout_keep_prob
                }
                _, step, summaries, loss, accuracy = sess.run(
                    [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                    feed_dict)
                time_str = datetime.datetime.now().isoformat()
                print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
                train_summary_writer.add_summary(summaries, step)

            def dev_step(x_batch, y_batch, writer=None):
                """
                Evaluates model on a dev set
                """
                feed_dict = {
                  cnn.input_x: x_batch,
                  cnn.input_y: y_batch,
                  cnn.dropout_keep_prob: 1.0
                }
                step, summaries, loss, accuracy = sess.run(
                    [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                    feed_dict)
                time_str = datetime.datetime.now().isoformat()
                print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
                if writer:
                    writer.add_summary(summaries, step)

            # Generate batches
            batches = data_helpers.batch_iter(
                list(zip(x_train, y_train)), FLAGS.batch_size, FLAGS.num_epochs)
            # Training loop. For each batch...
            for batch in batches:
                x_batch, y_batch = zip(*batch)
                train_step(x_batch, y_batch)
                current_step = tf.train.global_step(sess, global_step)
                if current_step % FLAGS.evaluate_every == 0:
                    print("\nEvaluation:")
                    dev_step(x_dev, y_dev, writer=dev_summary_writer)
                    print("")
                if current_step % FLAGS.checkpoint_every == 0:
                    path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                    print("Saved model checkpoint to {}\n".format(path))

In [4]:
# FLAGS = tf.flags.FLAGS

for i in range (10):

    positive_data_file = "data101617/dataNoClassCategory-" + str(i) + ".txt"
    negative_data_file = "data101617/dataNoClassNegCategory-" + str(i) + ".txt"
    
    train(i, positive_data_file, negative_data_file)

Loading data...
Vocabulary Size: 18248
Train/Dev split: 1140/60
INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/hist is illegal; using conv-maxpool-3/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/sparsity is illegal; using conv-maxpool-3/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/hist is illegal; using conv-maxpool-3/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/sparsity is illegal; using conv-maxpool-3/b_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/hist is illegal; using conv-maxpool-4/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/sparsity is illegal; using conv-maxpool-4/W_0/grad/sparsity instead.
INFO:tensorf

2017-11-05T07:53:50.148604: step 96, loss 0.856746, acc 0.84375
2017-11-05T07:53:54.802911: step 97, loss 0.357446, acc 0.875
2017-11-05T07:53:59.579084: step 98, loss 1.43444, acc 0.78125
2017-11-05T07:54:04.368486: step 99, loss 0.673507, acc 0.8125
2017-11-05T07:54:08.923222: step 100, loss 0.775227, acc 0.8125

Evaluation:
2017-11-05T07:54:12.212063: step 100, loss 1.39585, acc 0.833333

Saved model checkpoint to D:\ITB\S2\TESIS\cnn\runs\0\1509842765\checkpoints\model-100

2017-11-05T07:54:21.118976: step 101, loss 0.23491, acc 0.9375
2017-11-05T07:54:26.518351: step 102, loss 0.56634, acc 0.84375
2017-11-05T07:54:31.240205: step 103, loss 0.0519857, acc 0.96875
2017-11-05T07:54:35.872497: step 104, loss 0.778303, acc 0.9375
2017-11-05T07:54:40.756967: step 105, loss 0.479218, acc 0.9375
2017-11-05T07:54:45.876605: step 106, loss 0.917351, acc 0.78125
2017-11-05T07:54:50.452357: step 107, loss 0.739332, acc 0.9375
2017-11-05T07:54:53.450487: step 108, loss 1.1575, acc 0.75
2017-11-

2017-11-05T08:03:22.153150: step 220, loss 0.227559, acc 0.90625
2017-11-05T08:03:26.655850: step 221, loss 0.716176, acc 0.84375
2017-11-05T08:03:31.385210: step 222, loss 0.371057, acc 0.875
2017-11-05T08:03:35.914929: step 223, loss 0.455666, acc 0.875
2017-11-05T08:03:40.480411: step 224, loss 1.10929, acc 0.875
2017-11-05T08:03:44.950587: step 225, loss 0.514334, acc 0.875
2017-11-05T08:03:49.436775: step 226, loss 0.329683, acc 0.9375
2017-11-05T08:03:54.009595: step 227, loss 0.286685, acc 0.875
2017-11-05T08:03:58.511307: step 228, loss 0.316821, acc 0.90625
2017-11-05T08:04:03.023513: step 229, loss 0.735054, acc 0.8125
2017-11-05T08:04:07.587269: step 230, loss 0.435044, acc 0.96875
2017-11-05T08:04:12.167034: step 231, loss 0.257733, acc 0.90625
2017-11-05T08:04:16.654222: step 232, loss 0.794533, acc 0.78125
2017-11-05T08:04:21.223534: step 233, loss 1.05216, acc 0.78125
2017-11-05T08:04:25.776769: step 234, loss 0.192009, acc 0.9375
2017-11-05T08:04:30.324000: step 235, lo

2017-11-05T08:13:00.860484: step 346, loss 0.878916, acc 0.84375
2017-11-05T08:13:05.460753: step 347, loss 1.11071, acc 0.8125
2017-11-05T08:13:09.695762: step 348, loss 1.14431, acc 0.84375
2017-11-05T08:13:14.512654: step 349, loss 0.686643, acc 0.8125
2017-11-05T08:13:19.121012: step 350, loss 0.901567, acc 0.875
2017-11-05T08:13:23.394197: step 351, loss 0.440077, acc 0.875
2017-11-05T08:13:27.846560: step 352, loss 1.14401, acc 0.8125
2017-11-05T08:13:32.406835: step 353, loss 0.508842, acc 0.875
2017-11-05T08:13:36.516281: step 354, loss 0.232061, acc 0.9375
2017-11-05T08:13:40.860172: step 355, loss 0.165082, acc 0.9375
2017-11-05T08:13:44.884217: step 356, loss 0.222663, acc 0.90625
2017-11-05T08:13:48.972658: step 357, loss 0.388958, acc 0.90625
2017-11-05T08:13:53.145287: step 358, loss 0.348915, acc 0.8125
2017-11-05T08:13:57.311519: step 359, loss 0.374386, acc 0.875
2017-11-05T08:14:00.180401: step 360, loss 0.332965, acc 0.9
2017-11-05T08:14:05.524949: step 361, loss 0.3

2017-11-05T08:22:38.468934: step 472, loss 0.516312, acc 0.90625
2017-11-05T08:22:42.570900: step 473, loss 0.71017, acc 0.875
2017-11-05T08:22:46.849698: step 474, loss 0.176538, acc 0.96875
2017-11-05T08:22:50.922744: step 475, loss 0.374337, acc 0.84375
2017-11-05T08:22:55.116427: step 476, loss 0.626938, acc 0.9375
2017-11-05T08:22:59.500042: step 477, loss 0.0791407, acc 0.9375
2017-11-05T08:23:03.544753: step 478, loss 0.487113, acc 0.90625
2017-11-05T08:23:07.658638: step 479, loss 0.546769, acc 0.875
2017-11-05T08:23:11.849017: step 480, loss 0.456606, acc 0.9375
2017-11-05T08:23:15.997291: step 481, loss 0.51051, acc 0.875
2017-11-05T08:23:20.062033: step 482, loss 0.696208, acc 0.875
2017-11-05T08:23:24.187590: step 483, loss 0.734337, acc 0.84375
2017-11-05T08:23:28.799509: step 484, loss 0.786304, acc 0.875
2017-11-05T08:23:32.875961: step 485, loss 0.283144, acc 0.90625
2017-11-05T08:23:36.868022: step 486, loss 0.277393, acc 0.9375
2017-11-05T08:23:40.965597: step 487, lo

...

2017-11-06T23:47:50.770900: step 3096, loss 0.401297, acc 0.8
2017-11-06T23:47:54.696690: step 3097, loss 0.263115, acc 0.875
2017-11-06T23:47:58.704537: step 3098, loss 0.0910193, acc 0.9375
2017-11-06T23:48:02.767427: step 3099, loss 0.011269, acc 1
2017-11-06T23:48:06.799290: step 3100, loss 0.0528492, acc 1

Evaluation:
2017-11-06T23:48:09.352104: step 3100, loss 0.956632, acc 0.75

Saved model checkpoint to D:\ITB\S2\TESIS\cnn\runs\9\1509974316\checkpoints\model-3100

2017-11-06T23:48:14.728610: step 3101, loss 0.180423, acc 0.90625
2017-11-06T23:48:18.705435: step 3102, loss 0.132307, acc 0.96875
2017-11-06T23:48:22.881422: step 3103, loss 0.268619, acc 0.875
2017-11-06T23:48:26.954316: step 3104, loss 0.147869, acc 0.9375
2017-11-06T23:48:30.932141: step 3105, loss 0.0720589, acc 0.96875
2017-11-06T23:48:35.137130: step 3106, loss 0.240355, acc 0.90625
2017-11-06T23:48:39.109954: step 3107, loss 0.185937, acc 0.90625
2017-11-06T23:48:43.105792: step 3108, loss 0.0806801, acc 0.9

2017-11-06T23:56:04.119872: step 3217, loss 0.285274, acc 0.875
2017-11-06T23:56:08.091695: step 3218, loss 0.242861, acc 0.875
2017-11-06T23:56:12.107550: step 3219, loss 0.0785174, acc 0.9375
2017-11-06T23:56:16.114396: step 3220, loss 0.11173, acc 0.9375
2017-11-06T23:56:20.143259: step 3221, loss 0.221908, acc 0.90625
2017-11-06T23:56:24.127088: step 3222, loss 0.0934554, acc 0.9375
2017-11-06T23:56:28.643298: step 3223, loss 0.28963, acc 0.875
2017-11-06T23:56:32.977377: step 3224, loss 0.0621238, acc 0.96875
2017-11-06T23:56:37.194373: step 3225, loss 0.256465, acc 0.875
2017-11-06T23:56:41.255259: step 3226, loss 0.165863, acc 0.96875
2017-11-06T23:56:45.245094: step 3227, loss 0.0269174, acc 1
2017-11-06T23:56:49.274760: step 3228, loss 0.3158, acc 0.84375
2017-11-06T23:56:53.329641: step 3229, loss 0.232053, acc 0.9375
2017-11-06T23:56:57.344495: step 3230, loss 0.154571, acc 0.9375
2017-11-06T23:57:01.470426: step 3231, loss 0.191423, acc 0.90625
2017-11-06T23:57:05.558330: s

2017-11-07T00:04:35.885727: step 3341, loss 0.0682701, acc 0.96875
2017-11-07T00:04:40.761191: step 3342, loss 0.318846, acc 0.90625
2017-11-07T00:04:45.648665: step 3343, loss 0.228955, acc 0.875
2017-11-07T00:04:50.448074: step 3344, loss 0.225128, acc 0.90625
2017-11-07T00:04:55.149414: step 3345, loss 0.102955, acc 0.9375
2017-11-07T00:04:59.860762: step 3346, loss 0.308044, acc 0.875
2017-11-07T00:05:04.470037: step 3347, loss 0.248639, acc 0.90625
2017-11-07T00:05:07.247011: step 3348, loss 0.132271, acc 0.85
2017-11-07T00:05:11.819259: step 3349, loss 0.0535419, acc 0.96875
2017-11-07T00:05:16.529606: step 3350, loss 0.0676004, acc 0.96875
2017-11-07T00:05:20.689562: step 3351, loss 0.0696388, acc 0.96875
2017-11-07T00:05:25.107702: step 3352, loss 0.118773, acc 0.9375
2017-11-07T00:05:29.540852: step 3353, loss 0.391324, acc 0.8125
2017-11-07T00:05:33.979006: step 3354, loss 0.242333, acc 0.875
2017-11-07T00:05:38.286065: step 3355, loss 0.284573, acc 0.9375
2017-11-07T00:05:42

2017-11-07T00:13:08.957555: step 3465, loss 0.139893, acc 0.90625
2017-11-07T00:13:12.875339: step 3466, loss 0.0857062, acc 0.96875
2017-11-07T00:13:16.853166: step 3467, loss 0.28057, acc 0.875
2017-11-07T00:13:20.850007: step 3468, loss 0.108653, acc 0.96875
2017-11-07T00:13:24.905889: step 3469, loss 0.126923, acc 0.9375
2017-11-07T00:13:29.184928: step 3470, loss 0.201179, acc 0.90625
2017-11-07T00:13:33.132733: step 3471, loss 0.309095, acc 0.875
2017-11-07T00:13:37.076535: step 3472, loss 0.0705928, acc 0.9375
2017-11-07T00:13:41.038351: step 3473, loss 0.208421, acc 0.9375
2017-11-07T00:13:45.002167: step 3474, loss 0.16096, acc 0.90625
2017-11-07T00:13:48.971987: step 3475, loss 0.284774, acc 0.84375
2017-11-07T00:13:52.924797: step 3476, loss 0.0947229, acc 0.96875
2017-11-07T00:13:57.180820: step 3477, loss 0.142329, acc 0.9375
2017-11-07T00:14:01.406823: step 3478, loss 0.408273, acc 0.8125
2017-11-07T00:14:05.372641: step 3479, loss 0.00605733, acc 1
2017-11-07T00:14:09.34

2017-11-07T00:21:27.254642: step 3589, loss 0.151909, acc 0.96875
2017-11-07T00:21:31.178430: step 3590, loss 0.156224, acc 0.9375
2017-11-07T00:21:35.270338: step 3591, loss 0.107172, acc 0.96875
2017-11-07T00:21:39.636440: step 3592, loss 0.280706, acc 0.875
2017-11-07T00:21:43.626275: step 3593, loss 0.276355, acc 0.9375
2017-11-07T00:21:47.560070: step 3594, loss 0.0278866, acc 1
2017-11-07T00:21:51.489863: step 3595, loss 0.103432, acc 0.96875
2017-11-07T00:21:55.478697: step 3596, loss 0.29396, acc 0.78125
2017-11-07T00:21:59.467533: step 3597, loss 0.208032, acc 0.9375
2017-11-07T00:22:03.444358: step 3598, loss 0.144037, acc 0.96875
2017-11-07T00:22:07.438195: step 3599, loss 0.398014, acc 0.8125
2017-11-07T00:22:09.938971: step 3600, loss 0.475766, acc 0.8

Evaluation:
2017-11-07T00:22:12.500792: step 3600, loss 1.06058, acc 0.75

Saved model checkpoint to D:\ITB\S2\TESIS\cnn\runs\9\1509974316\checkpoints\model-3600



In [22]:
with tf.Session() as sess:    
    saver = tf.train.import_meta_graph('D:/ITB/S2/TESIS/cnn/runs/0/051117/checkpoints/model-3200.meta')
    saver.restore(sess, 'D:/ITB/S2/TESIS/cnn/runs/0/051117/checkpoints/model-3200')
    print(sess.run('w1:0'))
##Model has been restored. Above statement will print the saved value of w1.

INFO:tensorflow:Restoring parameters from D:/ITB/S2/TESIS/cnn/runs/0/051117/checkpoints/model-3200


ValueError: Fetch argument 'w1:0' cannot be interpreted as a Tensor. ("The name 'w1:0' refers to a Tensor which does not exist. The operation, 'w1', does not exist in the graph.")